In [35]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

from importlib import reload
import pathlib
import utils
import data
import shutil
import lesion_data

reload(utils);
reload(data);
reload(lesion_data);

In [2]:
import lesion_data
from lesion_data import LesionData
from importlib import reload
reload(lesion_data);


data = LesionData()
#data.download()
data.unzip(100)


Unzipping isic_data/ISIC_2019_Training_Input.zip
Unzipping isic_data/ISIC_2019_Test_Input.zip

Done unzipping file isic_data/ISIC_2019_Test_Input.zip
Done unzipping file isic_data/ISIC_2019_Training_Input.zip


In [43]:
# Loading data the keras way
data_dir = pathlib.Path("/".join([data.dir_name, 'ISIC_2019_Training_Input_']))

# nit clean up
try:
    shutil.rmtree(data.dir_name + "/ISIC_2019_Training_Input_/" + ".ipynb_checkpoints")
except OSError as e:
    pass

batch_size = 32
img_height = 244
img_width = 244
random_seed=123

train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=random_seed,
    image_size=(img_height, img_width),  # default 256x256
    batch_size=batch_size,
    shuffle=False,
)


AttributeError: 'LesionData' object has no attribute 'dir_name'

In [ ]:
train_ds

In [ ]:
# Print all classes found 
class_names = train_ds.class_names
print(class_names)

In [ ]:
# Caching 
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)

In [ ]:
# Layers 
num_classes = len(class_names)
learning_rate = 0.01

model = Sequential([
    layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        units=num_classes,
        activation='softmax',
        use_bias=True,
        kernel_initializer=tf.ones_initializer
    ),
])

optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    
model.compile(optimizer=optimizer,
              loss=tf.keras.losses.SparseCategoricalCrossentropy,
              metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_ds,
    epochs=30,
    batch_size=10
)